Questão 3: Trabalhando com Vector Stores e Embeddings
Você deve criar um sistema de busca semântica de documentos utilizando  embeddings e vector stores. Para isso, siga as etapas abaixo:
1. Utilize um conjunto de documentos de texto (pode ser um conjunto de artigos ou posts de um blog).
2. Gere embeddings para esses documentos utilizando um modelo de embeddings.
3. Armazene esses embeddings em uma vector store como FAISS ou Milvus.
4. Implemente uma função de busca que, dado um texto de consulta,retorne os documentos mais relevantes com base na similaridade semântica.

Dicas:

*   Utilize bibliotecas como transformers para gerar embeddings.
*   Documente o processo de criação dos embeddings e armazenamento na vector
store.
*   Demonstre a busca semântica com exemplos de consultas e resultados relevantes.

In [ ]:
!pip install rank-bm25 pypdf bitsandbytes sentence-transformers torch faiss-cpu transformers langchain langchain_community

### Observação: utilizar GPU - colab Pro

### Documento: 200 Receitas União

In [ ]:
!wget -O livro_receitas.pdf https://uniao.com.br/public/_assets/livros/200-1-volume.pdf

--2025-11-23 01:37:39--  https://uniao.com.br/public/_assets/livros/200-1-volume.pdf
Resolving uniao.com.br (uniao.com.br)... 13.35.185.17, 13.35.185.18, 13.35.185.66, ...
Connecting to uniao.com.br (uniao.com.br)|13.35.185.17|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15819225 (15M) [application/pdf]
Saving to: ‘livro_receitas.pdf’

livro_receitas.pdf  100%[===================>]  15.09M  6.58MB/s    in 2.3s    

2025-11-23 01:37:42 (6.58 MB/s) - ‘livro_receitas.pdf’ saved [15819225/15819225]



In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_community.vectorstores import FAISS

# carregar o Livro de Receitas
loader = PyPDFLoader("livro_receitas.pdf")
documentos = loader.load()
print(f"Total de páginas: {len(documentos)}")

from langchain_text_splitters import RecursiveCharacterTextSplitter

# chunks cada página
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=600,
    chunk_overlap=200
)

textos = text_splitter.split_documents(documentos)

#modelo de embeddings
embeddings = SentenceTransformerEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

# gerar o vector_store e salvar local
vector_store = FAISS.from_documents(textos, embeddings)
vector_store.save_local("./faiss_receitas")


Total de páginas: 306


####  busca semântica:

In [ ]:
# Busca por similaridade
query = "castanhas portuguesas"
results = vector_store.similarity_search(query, k=8)
for doc in results:
      print(f"Pág-> {doc.metadata.get("page")}  |  Trecho: {doc.page_content[:200]!r}")

Pág-> 232  |  Trecho: 'V K\ntortas em camadas\nTorta de Zug com kirsch.................................................................... 233\nTorta de ameixas e amêndoas.......................................................'
Pág-> 232  |  Trecho: 'Torta de castanhas de caju ................................................................. 237\nPão-de-ló recheado............................................................................. 238\nTor'
Pág-> 237  |  Trecho: 'V!V\ntorta de castanhas de caju\nH•« «•IIii (n 26-A) enviada por Da. Helena Luciano, residente à rua Honduras  \no 199 .Id. Paulista — São Paulo — SP\nINGREDIENTES — MASSA\nr. colheres de sopa de açúcar 25'
Pág-> 297  |  Trecho: 'llolo Mat i.i I ui/a ..................................................................................... 21\nllolo vicncnsc de mel.....................................................................'
Pág-> 14  |  Trecho: 'Bolo de castanhas portuguesas .....................................

In [ ]:
# Busca por similaridade com score
query = "castanhas portuguesas"
results = vector_store.similarity_search_with_score(query, k=5)

for doc, score in results:
      print(f"Pág-> {doc.metadata.get("page")} | SCORE: {score:.4f}  |  Trecho: {doc.page_content[:200]!r}")

Pág-> 232 | SCORE: 0.9899  |  Trecho: 'V K\ntortas em camadas\nTorta de Zug com kirsch.................................................................... 233\nTorta de ameixas e amêndoas.......................................................'
Pág-> 232 | SCORE: 1.0646  |  Trecho: 'Torta de castanhas de caju ................................................................. 237\nPão-de-ló recheado............................................................................. 238\nTor'
Pág-> 237 | SCORE: 1.0914  |  Trecho: 'V!V\ntorta de castanhas de caju\nH•« «•IIii (n 26-A) enviada por Da. Helena Luciano, residente à rua Honduras  \no 199 .Id. Paulista — São Paulo — SP\nINGREDIENTES — MASSA\nr. colheres de sopa de açúcar 25'
Pág-> 297 | SCORE: 1.0942  |  Trecho: 'llolo Mat i.i I ui/a ..................................................................................... 21\nllolo vicncnsc de mel.....................................................................'
Pág-> 14 | SCORE: 1.0958 

#### **OBS:** Para a query, a resposta com o SCORE mais próxima de 1 estava errada. Já a resposta correta "Receita (n° 155-A)", conforme o livro de receitas, teve um SCORE mais distante de 1, indicando um falso positivo.

###  Fine Tuning:
O Objetico é deixa "Receita (n° 155-A)-bolo de castanhas portuguesas" melhor classificada ou seja mais próxima de 1

In [ ]:
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever

# bm25
bm25_retriever = BM25Retriever.from_documents(textos)
bm25_retriever.k = 5

# dense
dense_retriever = vector_store.as_retriever(search_kwargs={"k": 5})

# combinando e informando os pesos
retriever_hibrido = EnsembleRetriever(
    retrievers=[bm25_retriever, dense_retriever],
    weights=[0.5, 0.5]   # você pode ajustar os pesos
)


In [ ]:
query = "bolo de castanhas portuguesas"

print(f"Páginas com a query: {query} \n")
resultados = retriever_hibrido.invoke(query)
for r in resultados:
  print(f"Pág-> {r.metadata.get("page")} | Trecho: {r.page_content[:200]!r}")



Páginas com a query: bolo de castanhas portuguesas 

Pág-> 14 | Trecho: 'Bolo de castanhas portuguesas .......................................................... 23\nDelicioso bolo de fru ta s ...................................................................... 24\nBolo in'
Pág-> 14 | Trecho: 'Bolo de coco e ameixas........................................................................ 19\nBolo Dona A parecida .......................................................................... 20\nBol'
Pág-> 30 | Trecho: 'Bolo de Maçãs e Nozes'
Pág-> 23 | Trecho: 'V!\nbolo de castanhas portuguesas\nReceita (n° 155-A) enviada por Da. Maria Cândida Rodrigues Dupont, residente à  \navenida Indianópolis n° 1.500 — São Paulo — SP\nINGREDIENTES — MASSA\n400g de açúcar UNI'
Pág-> 29 | Trecho: 'Bolo Supetjino de Fubá'
Pág-> 297 | Trecho: 'llolo Mat i.i I ui/a ..................................................................................... 21\nllolo vicncnsc de mel................................

#### **OBS:** Com relação ao "vector_store.similarity_search_with_score" houve uma melhorar siginificativa provando a eficiencia do Fine Tuning.

In [ ]:
# Metodo feito para ser reaproveitado no RAG
def busca_retriever_hibrido_com_score(query, threshold = 0.35):
    '''
     Customizando o retriever_hibrido add score e retormar
     apenas etonar apenas acima do threshold
    '''

    resultados = retriever_hibrido.invoke(query)

    embedding_query = embeddings.embed_query(query)
    embeddings_docs = embeddings.embed_documents([doc.page_content for doc in resultados])
    import numpy as np
    # Cálculo vetorizado
    embedding_query_arr = np.array(embedding_query)
    embeddings_docs_arr = np.array(embeddings_docs)

    norms_query = np.linalg.norm(embedding_query_arr)
    norms_docs = np.linalg.norm(embeddings_docs_arr, axis=1)
    scores = np.dot(embeddings_docs_arr, embedding_query_arr) / (norms_query * norms_docs)

    for doc, score in zip(resultados, scores):
            doc.metadata["score"] = float(score)

    filtrados = [d for d in resultados if d.metadata["score"] >= threshold]
    return sorted(filtrados, key=lambda x: x.metadata["score"], reverse=True)

### Teste do do threshold -> fora do contexto não retorna nenhum documento

In [ ]:
query = "Recife "
print(f"Páginas com a query: {query}")
resultados = busca_retriever_hibrido_com_score(query)
if not resultados:
  print("Nenhum resultado encontrado!!!")
else:
    for r in resultados:
      print(f"-> Pág:{r.metadata.get("page")} | score={r.metadata['score']:.3f}")

Páginas com a query: Recife 
Nenhum resultado encontrado!!!


### Teste do limite (threshold) -> retormar apenas retonar apenas acima do threshold

In [ ]:
query = "bolo de castanhas portuguesas"
print(f"Páginas com a query: {query}")
resultados = busca_retriever_hibrido_com_score(query)
if not resultados:
  print("Nenhum resultado encontrado.")
else:
    for r in resultados:
      print(f"-> Pág:{r.metadata.get("page")} | score={r.metadata['score']:.3f} | Trecho: {r.page_content[:200]!r}")

Páginas com a query: bolo de castanhas portuguesas
-> Pág:14 | score=0.630 | Trecho: 'Bolo de castanhas portuguesas .......................................................... 23\nDelicioso bolo de fru ta s ...................................................................... 24\nBolo in'
-> Pág:30 | score=0.619 | Trecho: 'Bolo de Maçãs e Nozes'
-> Pág:29 | score=0.606 | Trecho: 'Bolo Supetjino de Fubá'
-> Pág:14 | score=0.534 | Trecho: 'Bolo de coco e ameixas........................................................................ 19\nBolo Dona A parecida .......................................................................... 20\nBol'
-> Pág:14 | score=0.531 | Trecho: 'y V!\nbolos\nBolo de n atal.........................................................................................  15\nBolo de natal de tâmaras e nozes.................................................'
-> Pág:23 | score=0.487 | Trecho: 'V!\nbolo de castanhas portuguesas\nReceita (n° 155-A) enviada por Da. Maria Cân

In [ ]:
query = "Receita (n° 155-A)"
print(f"Páginas com a query: {query}")
resultados = busca_retriever_hibrido_com_score(query)
if not resultados:
  print("Nenhum resultado encontrado.")
else:
    for r in resultados:
      print(f"-> Pág:{r.metadata.get("page")} | score={r.metadata['score']:.3f} | Trecho: {r.page_content[:200]!r}")

Páginas com a query: Receita (n° 155-A)
-> Pág:6 | score=0.467 | Trecho: 'DOCELAR\nlUNlAO)\nS ___r .\nCursos de Culinária  \nRua Oscar Freire n°. 1463 \nCEP. 05409 - SÃO PA ULO - SP'
-> Pág:217 | score=0.458 | Trecho: 'm a a que possa ser estendida com o rolo, com 1/2 cm de espessura. Forre uma \ntoium rcfratária c coloque o recheio.\nINGREDIENTES — RECHEIO\n\\ xicni as de chá de açúcar UNIÃO 2 gemas\n\\ Mearas de chá bem'
-> Pág:208 | score=0.433 | Trecho: 'Torta de pêras à francesa .................................................................. 215\nTorta de avelãs.................................................................................... 216'
-> Pág:302 | score=0.422 | Trecho: 'Torta de pêras à francesa................................................................... 215\nTorta de avelãs.................................................................................... 216'
-> Pág:45 | score=0.412 | Trecho: 'canudos de Chantilly\nReceita tn" 58-A) enviada por Da. Júli

#### **OBS:** Segunda a documentação cada retriever interno retorna seus próprios documentos e scores.
*   Ensemble soma ou faz média ponderada desses scores e ordena os documentos pelo score final (maior para  menor).
*   Depois retorna a lista final apenas baseada no score combinado.
*   A página '23' é onde está a "Receita (n° 155-A) interessante que "bolo de *" tem os melhores resultados